In [1]:
#Based off of TM-Evaluate-Models-Worksheet in that instead of getting a score for a test set we are printing bets for 
#a test set

In [2]:
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import GradientBoostingClassifier

In [3]:
import sys
sys.path.append('../automated_model_creation') #We need to access the function file

In [4]:
import pandas as pd
import numpy as np
from functions import custom_cv_eval, get_ev_from_df, get_bet_ev, get_bet_return
import random
import csv


In [5]:
df = pd.read_csv("../data/ufc-master.csv")
df_upcoming = pd.read_csv('../data/upcoming-event.csv')
len(df)

4292

In [6]:
num_upcoming_fights = len(df_upcoming)

In [7]:
df = df_upcoming.append(df)
len(df)

4307

In [8]:
#Let's put all the labels in a dataframe
df['label'] = ''
#If the winner is not Red or Blue we can remove it.
mask = df['Winner'] == 'Red'
df['label'][mask] = 0
mask = df['Winner'] == 'Blue'
df['label'][mask] = 1

#df["Winner"] = df["Winner"].astype('category')
#df = df[(df['Winner'] != 'Blue') | (df['Winner'] == 'Red') ]


#Make sure lable is numeric
df['label'] = pd.to_numeric(df['label'], errors='coerce')

D:\Anaconda3\lib\site-packages\ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """
D:\Anaconda3\lib\site-packages\ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys


In [9]:
#Let's fix the date
df['date'] = pd.to_datetime(df['date'])

In [10]:
#Create a label df:
label_df = df['label']

#Let's create an odds df too:
odds_df = df[['R_odds', 'B_odds']]

In [11]:
#Split the test set.  We are always(?) going to use the last 200 matches as the test set, so we don't want those around
#as we pick models

df_train = df[num_upcoming_fights:]
odds_train = odds_df[num_upcoming_fights:]
label_train = label_df[num_upcoming_fights:]

df_test = df[:num_upcoming_fights]
odds_test = odds_df[:num_upcoming_fights]
label_test = label_df[:num_upcoming_fights]

print(len(df_test))
print(len(odds_test))
print(len(label_test))

print(len(df_train))
print(len(odds_train))
print(len(label_train))

15
15
15
4292
4292
4292


In [12]:
display(df_train)
display(df_test)

,R_fighter,B_fighter,R_odds,B_odds,R_ev,B_ev,date,location,country,Winner,...,B_Welterweight_rank,B_Lightweight_rank,B_Featherweight_rank,B_Bantamweight_rank,B_Flyweight_rank,B_Pound-for-Pound_rank,better_rank,finish,finish_details,label
0,Deiveson Figueiredo,Joseph Benavidez,-225.0,180.0,44.4444,180,2020-07-18,"Abu Dhabi, Abu Dhabi, United Arab Emirates",United Arab Emirates,Red,...,NaN,NaN,NaN,NaN,2.0,NaN,Red,SUB,Rear Naked Choke,0.0
1,Jack Hermansson,Kelvin Gastelum,-112.0,-112.0,89.2857,89.2857,2020-07-18,"Abu Dhabi, Abu Dhabi, United Arab Emirates",United Arab Emirates,Red,...,NaN,NaN,NaN,NaN,NaN,NaN,Red,SUB,Heel Hook,0.0
2,Marc Diakiese,Rafael Fiziev,-167.0,135.0,59.8802,135,2020-07-18,"Abu Dhabi, Abu Dhabi, United Arab Emirates",United Arab Emirates,Blue,...,NaN,NaN,NaN,NaN,NaN,NaN,neither,U-DEC,NaN,1.0
3,Ariane Lipski,Luana Carolina,-125.0,100.0,80,100,2020-07-18,"Abu Dhabi, Abu Dhabi, United Arab Emirates",United Arab Emirates,Red,...,NaN,NaN,NaN,NaN,NaN,NaN,neither,SUB,Kneebar,0.0
4,Alexandre Pantoja,Askar Askarov,-210.0,165.0,47.619,165,2020-07-18,"Abu Dhabi, Abu Dhabi, United Arab Emirates",United Arab Emirates,Blue,...,NaN,NaN,NaN,NaN,NaN,NaN,neither,U-DEC,NaN,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4287,Duane Ludwig,Darren Elkins,-155.0,135.0,64.5161,135,2010-03-21,"Broomfield, Colorado, USA",USA,Blue,...,NaN,NaN,NaN,NaN,NaN,NaN,neither,KO/TKO,NaN,1.0
4288,John Howard,Daniel Roberts,-210.0,175.0,47.619,175,2010-03-21,"Broomfield, Colorado, USA",USA,Red,...,NaN,NaN,NaN,NaN,NaN,NaN,neither,KO/TKO,Punch,0.0
4289,Brendan Schaub,Chase Gormley,-260.0,220.0,38.4615,220,2010-03-21,"Broomfield, Colorado, USA",USA,Red,...,NaN,NaN,NaN,NaN,NaN,NaN,neither,KO/TKO,Punches,0.0
4290,Mike Pierce,Julio Paulino,-420.0,335.0,23.8095,335,2010-03-21,"Broomfield, Colorado, USA",USA,Red,...,NaN,NaN,NaN,NaN,NaN,NaN,neither,U-DEC,NaN,0.0


,R_fighter,B_fighter,R_odds,B_odds,R_ev,B_ev,date,location,country,Winner,...,B_Welterweight_rank,B_Lightweight_rank,B_Featherweight_rank,B_Bantamweight_rank,B_Flyweight_rank,B_Pound-for-Pound_rank,better_rank,finish,finish_details,label
0,Robert Whittaker,Darren Till,-120.0,-105.0,83.33333333,95.23809524,2020-07-25,"Abu Dhabi, Abu Dhabi, United Arab Emirates",United Arab Emirates,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,Red,NaN,NaN,NaN
1,Mauricio Rua,Rogerio Nogueira,-200.0,155.0,50,155,2020-07-25,"Abu Dhabi, Abu Dhabi, United Arab Emirates",United Arab Emirates,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,Red,NaN,NaN,NaN
2,Fabricio Werdum,Alexander Gustafsson,260.0,-335.0,260,29.85074627,2020-07-25,"Abu Dhabi, Abu Dhabi, United Arab Emirates",United Arab Emirates,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,Blue,NaN,NaN,NaN
3,Carla Esparza,Marina Rodriguez,134.0,-167.0,134,59.88023952,2020-07-25,"Abu Dhabi, Abu Dhabi, United Arab Emirates",United Arab Emirates,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,Red,NaN,NaN,NaN
4,Paul Craig,Gadzhimurad Antigulov,-112.0,-112.0,89.28571429,89.28571429,2020-07-25,"Abu Dhabi, Abu Dhabi, United Arab Emirates",United Arab Emirates,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,neither,NaN,NaN,NaN
5,Alex Oliveira,Peter Sobotta,-177.0,140.0,56.49717514,140,2020-07-25,"Abu Dhabi, Abu Dhabi, United Arab Emirates",United Arab Emirates,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,neither,NaN,NaN,NaN
6,Khamzat Chimaev,Rhys McKee,NaN,NaN,#DIV/0!,#DIV/0!,2020-07-25,"Abu Dhabi, Abu Dhabi, United Arab Emirates",United Arab Emirates,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,neither,NaN,NaN,NaN
7,Francisco Trinaldo,Jai Herbert,-177.0,140.0,56.49717514,140,2020-07-25,"Abu Dhabi, Abu Dhabi, United Arab Emirates",United Arab Emirates,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,neither,NaN,NaN,NaN
8,Nicolas Dalby,Jesse Ronson,-265.0,205.0,37.73584906,205,2020-07-25,"Abu Dhabi, Abu Dhabi, United Arab Emirates",United Arab Emirates,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,neither,NaN,NaN,NaN
9,Tom Aspinall,Jake Collier,-225.0,175.0,44.44444444,175,2020-07-25,"Abu Dhabi, Abu Dhabi, United Arab Emirates",United Arab Emirates,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,neither,NaN,NaN,NaN


In [13]:
def display_bets(ev_df, min_ev):
    
    for index, row in ev_df.iterrows():
        t1_bet_ev = get_bet_ev(row['t1_odds'], row['t1_prob'])
        t2_bet_ev = get_bet_ev(row['t2_odds'], row['t2_prob'])
        #print(f"ODDS:{row['t2_odds']} PROB: {row['t2_prob']} EV: {t2_bet_ev}")
        t1_bet_return = get_bet_return(row['t1_odds'])
        t2_bet_return = get_bet_return(row['t2_odds'])
        print(f"{row['t1_name']} vs. {row['t2_name']}")
        if t1_bet_ev > min_ev:
            print(f"{row['t1_name']} has an EV of {round(t1_bet_ev, 2)} on odds of {round(row['t1_odds'],2)}.  They have {round(row['t1_prob']*100,2)}", 
                  "% chance of winning.")
        
        elif t2_bet_ev > min_ev:
            print(f"{row['t2_name']} has an EV of {round(t2_bet_ev, 2)} on odds of {round(row['t2_odds'],2)}.  They have {round(row['t2_prob']*100,2)}", 
                  "% chance of winning.")
        
        else:
            print(f"No bets.  EV1:{t1_bet_ev}. EV2: {t2_bet_ev}")
            print(f"{row['t1_name']} has an EV of {round(t1_bet_ev, 2)} on odds of {round(row['t1_odds'],2)}.  They have {round(row['t1_prob']*100,2)}", 
                  "% chance of winning.")
        
        print()
        
    return(0)
        

In [14]:
def get_bets(input_model, input_features, input_ev, train_df, train_labels, train_odds, test_df, test_labels,
             test_odds, verbose=True):
    fighters_test = df[['R_fighter', 'B_fighter']]
    #Train model.  
    df_train = train_df[input_features].copy()
    df_test = test_df[input_features].copy()
    df_train = df_train.dropna()
    df_test = df_test.dropna()
        
    df_train = pd.get_dummies(df_train)
    df_test = pd.get_dummies(df_test)
    df_train, df_test = df_train.align(df_test, join='left', axis=1)    #Ensures both sets are dummified the same
    df_test = df_test.fillna(0)

    #LOOK AT get_ev and prepare the labels and odds
    
    labels_train = train_labels[train_labels.index.isin(df_train.index)]
    odds_train = train_odds[train_odds.index.isin(df_train.index)] 
    labels_test = test_labels[test_labels.index.isin(df_test.index)]
    odds_test = test_odds[test_odds.index.isin(df_test.index)] 
    fighters_test = fighters_test[fighters_test.index.isin(df_test.index)]

    input_model.fit(df_train, labels_train)
    
    probs = input_model.predict_proba(df_test)
    odds_test = np.array(odds_test)  
    fighters_test = np.array(fighters_test)
    #display(fighters_test)
    prepped_test = list(zip(fighters_test[:, -2], fighters_test[:, -1], odds_test[:, -2], odds_test[:, -1], 
                            probs[:, 0], probs[:, 1], labels_test))
    ev_prepped_df = pd.DataFrame(prepped_test, columns=['t1_name', 't2_name', 't1_odds', 't2_odds', 't1_prob', 't2_prob', 'winner'])
    
    #display(ev_prepped_df)
    
    display_bets(ev_prepped_df, input_ev)
    
    return(0)

In [15]:
with open('../automated_model_creation/models.csv', newline='') as f:
    reader = csv.reader(f)
    models = list(reader)
    
#print(len(models))

#set the model choice:
model_choice = 9

print(models)



model = eval(models[1][model_choice])
features = eval(models[2][model_choice])
ev = eval(models[3][model_choice])
print()
print(model, features, ev)

[['model_0', 'model_1', 'model_2', 'model_3', 'model_4', 'model_06_13_20', 'model_06_20_20', 'model_6_27_20', 'model_7_18_20', 'model_7_25_20'], ['LogisticRegression(max_iter=50000, random_state=75, tol=5.993308915320019)', 'DecisionTreeClassifier(max_depth=20, random_state=75)', "RandomForestClassifier(criterion='entropy', max_depth=10, n_estimators=150,\n                       random_state=75)", 'GradientBoostingClassifier(n_estimators=122, random_state=75)', 'GaussianNB()', 'LogisticRegression(max_iter=50000, random_state=75, tol=5.993308915320019)', 'GaussianNB()', 'GaussianNB()', 'GradientBoostingClassifier(n_estimators=122, random_state=75)', 'GaussianNB()'], ["['empty_arena', 'R_Weight_lbs', 'country', 'B_Weight_lbs', 'R_win_by_Decision_Split', 'R_draw', 'sig_str_dif', 'B_current_win_streak', 'B_Height_cms', 'R_win_by_TKO_Doctor_Stoppage', 'R_win_by_Submission', 'win_streak_dif', 'R_longest_win_streak', 'R_losses', 'B_longest_win_streak', 'R_total_title_bouts', 'R_current_win_st

In [16]:

get_bets(model, features, ev, df_train, label_train, odds_train, df_test, label_test,
                         odds_test, verbose = True)


Robert Whittaker vs. Darren Till
Darren Till has an EV of 95.24 on odds of -105.0.  They have 100.0 % chance of winning.

Mauricio Rua vs. Rogerio Nogueira
Mauricio Rua has an EV of 41.34 on odds of -200.0.  They have 94.23 % chance of winning.

Fabricio Werdum vs. Alexander Gustafsson
Fabricio Werdum has an EV of 168.72 on odds of 260.0.  They have 74.64 % chance of winning.

Carla Esparza vs. Marina Rodriguez
Marina Rodriguez has an EV of 59.88 on odds of -167.0.  They have 100.0 % chance of winning.

Paul Craig vs. Gadzhimurad Antigulov
Paul Craig has an EV of 72.29 on odds of -112.0.  They have 91.02 % chance of winning.

Alex Oliveira vs. Peter Sobotta
Alex Oliveira has an EV of 8.23 on odds of -177.0.  They have 69.16 % chance of winning.

Francisco Trinaldo vs. Jai Herbert
Francisco Trinaldo has an EV of 47.19 on odds of -177.0.  They have 94.05 % chance of winning.

Nicolas Dalby vs. Jesse Ronson
Nicolas Dalby has an EV of 2.67 on odds of -265.0.  They have 74.54 % chance of wi

0